<a href="https://colab.research.google.com/github/SAH-UJA/Video-Streaming-Using-Flask/blob/main/Video_Streaming.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!apt install libasound2-dev portaudio19-dev libportaudio2 libportaudiocpp0 ffmpeg
!pip install pyaudio

Reading package lists... Done
Building dependency tree       
Reading state information... Done
libasound2-dev is already the newest version (1.1.3-5ubuntu0.5).
ffmpeg is already the newest version (7:3.4.8-0ubuntu0.2).
The following package was automatically installed and is no longer required:
  libnvidia-common-440
Use 'apt autoremove' to remove it.
Suggested packages:
  portaudio19-doc
The following NEW packages will be installed:
  libportaudio2 libportaudiocpp0 portaudio19-dev
0 upgraded, 3 newly installed, 0 to remove and 35 not upgraded.
Need to get 184 kB of archives.
After this operation, 891 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libportaudio2 amd64 19.6.0-1 [64.6 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libportaudiocpp0 amd64 19.6.0-1 [15.1 kB]
Get:3 http://archive.ubuntu.com/ubuntu bionic/universe amd64 portaudio19-dev amd64 19.6.0-1 [104 kB]
Fetched 184 kB in 1s (258 kB/s)
Selecting pr

In [ ]:
import pyaudio

In [ ]:
!pip install flask-ngrok

In [ ]:
cd sample_data

[Errno 2] No such file or directory: 'sample_data'
/content/sample_data


In [ ]:
from moviepy.editor import *
video = VideoFileClip("vid.mp4")
audio = video.audio
audio.write_audiofile('audi.wav')

[MoviePy] Writing audio in audi.wav


100%|██████████| 126/126 [00:00<00:00, 1971.81it/s]

[MoviePy] Done.


In [ ]:
from flask import Flask, render_template, Response
from flask_ngrok import run_with_ngrok
import pyaudio
import cv2

FORMAT = pyaudio.paInt16
CHANNELS = 1
RATE = 44100
CHUNK = 1024

audio = pyaudio.PyAudio()


app = Flask(__name__)
run_with_ngrok(app)

@app.route('/')
def index():
    """Video streaming home page."""
    return render_template('index.html')




@app.route("/audio_feed")
def audio_feed():
    """Audio streaming route. Put this in the src attribute of an audio tag."""
    return Response(generateAudio(),
                    mimetype="audio/x-wav")

@app.route("/vaud_feed")
def vaud_feed():
  return Response(generateVaudio(), mimetype='video/mp4')

# Stream routing
@app.route('/video_feed')
def video_feed():
    """Video streaming route. Put this in the src attribute of an img tag."""
    return Response(generateVideo(),
                    mimetype='multipart/x-mixed-replace; boundary=frame')
# Stream generating
def generateVideo():
    """Video streaming generator function."""
    cam = cv2.VideoCapture('vid.mp4')
    while True:
        ret,frame = cam.read()
        frame = cv2.rotate(frame, cv2.ROTATE_90_CLOCKWISE)
        frame = cv2.resize(frame, (100,100), interpolation = cv2.INTER_AREA)
        gray = cv2.imencode('.jpg',frame)[1].tobytes()
        #gray = io.BytesIO(gray).getvalue()
        yield (b'--frame\r\n'+b'Content-Type: image/jpeg\r\n\r\n'+gray+b'\r\n')


def generateAudio():

    with open("audi.wav", "rb") as fwav:
      data = fwav.read(1024)
      while data:
        yield data
        data = fwav.read(1024)

def generateVaudio():

    with open("vid.mp4", "rb") as fwav:
      data = fwav.read(1024)
      while data:
        yield data
        data = fwav.read(1024)


def genHeader(sampleRate, bitsPerSample, channels, samples):
    datasize = samples * channels * bitsPerSample // 8
    o = bytes("RIFF",'ascii')                                               # (4byte) Marks file as RIFF
    o += (datasize + 36).to_bytes(4,'little')                               # (4byte) File size in bytes excluding this and RIFF marker
    o += bytes("WAVE",'ascii')                                              # (4byte) File type
    o += bytes("fmt ",'ascii')                                              # (4byte) Format Chunk Marker
    o += (16).to_bytes(4,'little')                                          # (4byte) Length of above format data
    o += (1).to_bytes(2,'little')                                           # (2byte) Format type (1 - PCM)
    o += (channels).to_bytes(2,'little')                                    # (2byte)
    o += (sampleRate).to_bytes(4,'little')                                  # (4byte)
    o += (sampleRate * channels * bitsPerSample // 8).to_bytes(4,'little')  # (4byte)
    o += (channels * bitsPerSample // 8).to_bytes(2,'little')               # (2byte)
    o += (bitsPerSample).to_bytes(2,'little')                               # (2byte)
    o += bytes("data",'ascii')                                              # (4byte) Data Chunk Marker
    o += (datasize).to_bytes(4,'little')                                    # (4byte) Data size in bytes
    return o
app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://3fc6c7a49d07.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


127.0.0.1 - - [13/Aug/2020 13:26:44] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [13/Aug/2020 13:26:45] "GET /video_feed HTTP/1.1" 200 -
127.0.0.1 - - [13/Aug/2020 13:26:46] "GET /audio_feed HTTP/1.1" 200 -
127.0.0.1 - - [13/Aug/2020 13:26:46] "GET /vaud_feed HTTP/1.1" 200 -
Error on request:
Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/werkzeug/serving.py", line 323, in run_wsgi
    execute(self.server.app)
  File "/usr/local/lib/python3.6/dist-packages/werkzeug/serving.py", line 314, in execute
    for data in application_iter:
  File "/usr/local/lib/python3.6/dist-packages/werkzeug/wsgi.py", line 506, in __next__
    return self._next()
  File "/usr/local/lib/python3.6/dist-packages/werkzeug/wrappers/base_response.py", line 45, in _iter_encoded
    for item in iterable:
  File "<ipython-input-59-cfe47751fb02>", line 48, in generateVideo
    
cv2.error: OpenCV(4.1.2) /io/opencv/modules/imgproc/src/resize.cpp:3720: error: (-215:Assertion failed) !ssize.em